In [1]:
import pandas as pd
import ast

In [2]:
df_transcripts = pd.read_json("transcript1.txt", orient="index")

df_transcripts = df_transcripts.drop(["altmetric_id", "abstract"], axis=1)

In [3]:
df_transcripts = df_transcripts.dropna()
ids_with_transcripts = df_transcripts.id.values

In [4]:
df = pd.read_csv("combined_with_news_n_blogs.csv", header=0)

df_id_video = pd.DataFrame(columns = ["altmetric_id", "video_id", "abstract", "transcript"])

df = df[df.link != '0']

In [6]:
i=0
for index, row in df.iterrows():
    video_ids = ast.literal_eval(row['link'])
    for v_id in video_ids:
        if v_id in ids_with_transcripts:
            df_id_video.loc[i, "video_id"] = v_id
            df_id_video.loc[i, "altmetric_id"] = row['altmetric_id']
            df_id_video.loc[i, "abstract"] = row["abstract"]
    i=i+1

In [7]:
for index, row in df_id_video.iterrows():
    df_id_video.loc[index, "transcript"] = df_transcripts.loc[df_transcripts.id == row["video_id"], "transcript"].values[0]

In [8]:
df_id_video = df_id_video[df_id_video.abstract != "0"]

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
def cosine_sim(d1, d2):
    tfidf = vectorizer.fit_transform([d1, d2])
    return ((tfidf * tfidf.T).A)[0,1]


In [10]:
df_id_video["score"] = 0
for index, row in df_id_video.iterrows():
    df_id_video.loc[index, "score"] = cosine_sim(row['abstract'], row['transcript'])

In [12]:
#scores with greater than 0.75
import numpy as np
len(np.where(df_id_video['score']>0.75)[0])

17

In [14]:
#scores with greater than 0.5 and less than 0.75
len(np.where(df_id_video['score']>0.50 )[0]) - len(np.where(df_id_video['score']>0.75 )[0])

2522

In [15]:
#scores with greater than 0.5 and less than 0.75
len(np.where(df_id_video['score']>0.25 )[0]) - len(np.where(df_id_video['score']>0.50 )[0])

9602

In [17]:
len(np.where(df_id_video['score']<=0.25)[0])

2138